# 공간 분석
> 공간 분석이란 지리공간정보를 데이터로 변환하여 특정 위치나 공간의 특징을 시각적으로 분석해내는 기법을 의미합니다.

## 목차
- import Base code
    - polygon / 폴리곤
    - heatmap / 히트맵
    - marker(point) / 마커(포인트)
    - bubble / 버블
- 함수 시연
    - polygon / 폴리곤
    - heatmap / 히트맵
    - marker(point) / 마커(포인트)
    - bubble / 버블

## import Base code

In [ ]:
! pip install geom==1.1

In [2]:
import json
import pandas as pd
import shapely.geometry

from Geom import search

def geojson2centroid(df):
    result = []
    for row in df.values.tolist():
        juso_code = search.juso_to_code(sido=row[1], sgg=row[2], emd=row[3])
        geojson = search.code_to_geom(juso_code=str(juso_code))
        result.append([
            row[0],
            shapely.geometry.shape(json.loads(geojson)).centroid.x,
            shapely.geometry.shape(json.loads(geojson)).centroid.y,
            row[4]]
        )
    return result

ModuleNotFoundError: No module named 'Geom'

## polygon / 폴리곤
> 폴리곤이란 지리공간정보를 지도 위에 표시하는 방식 중 하나로, 지도 위에 특정 포인트를 연결하여 생성한 다각형을 의미합니다.
지도 위에서 특정 포인트를 연결하는 것은 선분으로 나타납니다. 특정 범주에 속하는 인위적인 영역을 묶음 지어 표현하고자 할 때 주로 사용됩니다.

- 구성
    -  polygon 실행함수

In [3]:
# polygon 실행함수

def polygon(df: pd.DataFrame, geom_type):
    if geom_type == "주소":
        result = []
        for row in df.values.tolist():
            juso_code = search.juso_to_code(sido=row[1], sgg=row[2], emd=row[3])
            geojson = search.code_to_geom(juso_code=str(juso_code))
            result.append([row[0], geojson, row[4]])

        df = pd.DataFrame(
            data=result,
            columns=["LABEL", "GEOJSON", "COUNT"]
        )
    else:  # 위경도
        result = []
        for row in df.values.tolist():
            result.append([row[0], f'{{"type":"Point","coordinates":({row[1]}, {row[2]})}}', row[3]])

        df = pd.DataFrame(
            data=[],
            columns=["LABEL", "GEOJSON", "COUNT"]
        )
    return df

## heatmap / 히트맵
> 히트맵이란 지리공간정보를 지도 위에 표시하는 방식 중 하나로, Heat(열)와 Map(지도)의 합성어를 뜻합니다.

- 구성
    -  heatmap 실행함수

In [4]:
# heatmap 실행함수

def heatmap(df: pd.DataFrame, geom_type):
    if geom_type == "주소":
        result = geojson2centroid(df)
        df = pd.DataFrame(
            data=result,
            columns=["LABEL", "LAT", "LON", "COUNT"]
        )
    else:  # 위경도
        df.rename(columns={
            '라벨': 'LABEL', '위도': 'LAT', '경도': 'LON', '값(숫자)': 'COUNT'
        }, inplace=True)
    return df

## marker(point) / 마커(포인트)
> 마커란 지리공간정보를 지도 위에 표시하는 방식 중 하나로, 정해진 특정 포인트를 시각적으로 표현해내는 시각화 방법을 의미합니다.

- 구성
    -  marker(point) 실행함수

In [5]:
# marker(point) 실행함수

def point(df: pd.DataFrame, geom_type):
    if geom_type == "주소":
        result = geojson2centroid(df)
        df = pd.DataFrame(
            data=result,
            columns=["LABEL", "LAT", "LON", "COUNT"]
        )
    else:  # 위경도
        df.rename(columns={
            '라벨': 'LABEL', '위도': 'LAT', '경도': 'LON', '값(숫자)': 'COUNT'
        }, inplace=True)
    return df

## bubble / 버블
> 버블이란 지리공간정보를 지도 위에 표시하는 방식 중 하나로, 위도 및 경도로 기록된 지도 상의 위치에 특정 컬러로 칠해진 원형 도형을 통해 해당 위치에서의 측정값을 시각적으로 표현합니다.

- 구성
    -  bubble 실행함수

In [6]:
# bubble 실행함수

def bubble(df: pd.DataFrame, geom_type):
    if geom_type == "주소":
        result = geojson2centroid(df)
        df = pd.DataFrame(
            data=result,
            columns=["LABEL", "LAT", "LON", "COUNT"]
        )
    else:  # 위경도
        df.rename(columns={
            '라벨': 'LABEL', '위도': 'LAT', '경도': 'LON', '값(숫자)': 'COUNT'
        }, inplace=True)
    return df

# 함수 시연
## polygon / 폴리곤

In [7]:
column = [
    "라벨",	"주소_시도명",	"주소_시군구명",	"주소_읍면동명",	"가중치(숫자)"
]
value = [
["A",	"경상남도",	"거창군",	"신원면",	1498],
["B",	"전라남도",	"고흥군",	"도화면",	4411],
["C",	"대전광역시",	"서구",	"복수동",	8801]
]

address_df = pd.DataFrame(value, columns=column)

column = [
    "라벨",	"위도",	"경도",	"가중치(숫자)"
]
value = [
["인천국제공항",	337.4625,	126.439167,	14928],
["김포국제공항",	37.558056,	126.790556,	11568],
["제주국제공항",	33.511111,	126.492778,	13661],
["김해국제공항",	35.179444,	128.938056,	4626]
]

latlon_df = pd.DataFrame(value, columns=column)


address_polygon_df = polygon(address_df, "주소")
latlon_polygon_df = polygon(latlon_df, "위경도")
print(address_polygon_df)
print(latlon_polygon_df)

NameError: name 'search' is not defined

## heatmap / 히트맵

In [8]:
column = [
    "라벨",	"주소_시도명",	"주소_시군구명",	"주소_읍면동명",	"가중치(숫자)"
]
value = [
["A",	"경상남도",	"거창군",	"신원면",	1498],
["B",	"전라남도",	"고흥군",	"도화면",	4411],
["C",	"대전광역시",	"서구",	"복수동",	8801]
]

address_df = pd.DataFrame(value, columns=column)

column = [
    "라벨",	"위도",	"경도",	"가중치(숫자)"
]
value = [
["인천국제공항",	337.4625,	126.439167,	14928],
["김포국제공항",	37.558056,	126.790556,	11568],
["제주국제공항",	33.511111,	126.492778,	13661],
["김해국제공항",	35.179444,	128.938056,	4626]
]

latlon_df = pd.DataFrame(value, columns=column)


address_polygon_df = heatmap(address_df, "주소")
latlon_polygon_df = heatmap(latlon_df, "위경도")
print(address_polygon_df)
print(latlon_polygon_df)

NameError: name 'geojson2centroid' is not defined

## marker(point) / 마커(포인트)

In [9]:
column = [
    "라벨",	"주소_시도명",	"주소_시군구명",	"주소_읍면동명",	"가중치(숫자)"
]
value = [
["A",	"경상남도",	"거창군",	"신원면",	1498],
["B",	"전라남도",	"고흥군",	"도화면",	4411],
["C",	"대전광역시",	"서구",	"복수동",	8801]
]

address_df = pd.DataFrame(value, columns=column)

column = [
    "라벨",	"위도",	"경도",	"가중치(숫자)"
]
value = [
["인천국제공항",	337.4625,	126.439167,	14928],
["김포국제공항",	37.558056,	126.790556,	11568],
["제주국제공항",	33.511111,	126.492778,	13661],
["김해국제공항",	35.179444,	128.938056,	4626]
]

latlon_df = pd.DataFrame(value, columns=column)


address_polygon_df = point(address_df, "주소")
latlon_polygon_df = point(latlon_df, "위경도")
print(address_polygon_df)
print(latlon_polygon_df)

NameError: name 'geojson2centroid' is not defined

## bubble / 버블

In [10]:
column = [
    "라벨",	"주소_시도명",	"주소_시군구명",	"주소_읍면동명",	"가중치(숫자)"
]
value = [
["A",	"경상남도",	"거창군",	"신원면",	1498],
["B",	"전라남도",	"고흥군",	"도화면",	4411],
["C",	"대전광역시",	"서구",	"복수동",	8801]
]

address_df = pd.DataFrame(value, columns=column)

column = [
    "라벨",	"위도",	"경도",	"가중치(숫자)"
]
value = [
["인천국제공항",	337.4625,	126.439167,	14928],
["김포국제공항",	37.558056,	126.790556,	11568],
["제주국제공항",	33.511111,	126.492778,	13661],
["김해국제공항",	35.179444,	128.938056,	4626]
]

latlon_df = pd.DataFrame(value, columns=column)


address_polygon_df = bubble(address_df, "주소")
latlon_polygon_df = bubble(latlon_df, "위경도")
print(address_polygon_df)
print(latlon_polygon_df)

NameError: name 'geojson2centroid' is not defined